<a href="https://colab.research.google.com/github/allnes/age_classifier/blob/master/train_age_pip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Google Drive

In [13]:
from google.colab import drive
import os, natsort as nsrt, numpy as np, re
from scipy.sparse import csr_matrix, csgraph
import matplotlib.pyplot as plt
!pip install ChebyGCN

drive.mount('/content/drive')
PATH_PROJECT='/content/drive/My Drive/DL_DATA_GRAPH/'
PATH_CNN_REPO=PATH_PROJECT + 'BUILD/cnn_graph/'
os.chdir(PATH_CNN_REPO)
from lib import models, graph, coarsening, utils
%ls

# !git clone https://github.com/mdeff/cnn_graph
!git pull origin master

os.chdir(PATH_PROJECT)
%ls
%matplotlib inline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
lib/         makefile   rcv1.ipynb  requirements.txt  usage.ipynb
LICENSE.txt  nips2016/  README.md   trials/
From https://github.com/mdeff/cnn_graph
 * branch            master     -> FETCH_HEAD
Already up to date.
BUILD/  DATA/


## Preprocessing data

In [14]:
PATH_GRAPHS=PATH_PROJECT + 'DATA/mini_graphs/graphs/'
list_grpahs = []
for (_, _, filenames) in os.walk(PATH_GRAPHS):
    list_grpahs = list_grpahs + filenames

list_grpahs = nsrt.natsorted(list_grpahs)[0::2]

num_samples = int(np.load(PATH_GRAPHS + list_grpahs[0])['num_samples'])
num_features = int(np.load(PATH_GRAPHS + list_grpahs[0])['num_features'])

X = []
for graph_name in list_grpahs:
    raw_graph = np.load(PATH_GRAPHS + graph_name)
    raw_edges = raw_graph['E'].transpose()
    rows = raw_edges[0]
    cols = raw_edges[1]
    raw_data = raw_graph['D']
    sparse_graph = csr_matrix((raw_data, (rows, cols)),
                              shape=(raw_graph['num_features'],
                                     raw_graph['num_features']))
    X.append(sparse_graph)


PATH_LABELS=PATH_PROJECT + 'DATA/mini_graphs/GSE87571_samples.txt'

raw_file = open(PATH_LABELS, 'r')
y = []
for line in raw_file.readlines():
    match_obj = re.match(r'(GSM[0-9]*)\s*([M,F])\s*([0-9]*)\s*([0-9]*)', line)
    if not match_obj is None: 
        y.append(int(match_obj.group(3)))

assert len(y) == num_samples
assert len(X) == num_samples

X = np.array(X)
y = np.array(y)

print(raw_graph.files)
print(X.shape)
print(y.shape)

['M', 'D', 'E', 'num_samples', 'num_features']
(729,)
(729,)


## Train

In [20]:
from ChebyGCN import layers, coarsening

n_train = num_samples // 2
n_val = num_samples // 10

X_train = X[:n_train]
X_val   = X[n_train:n_train+n_val]
X_test  = X[n_train+n_val:]

y_train = y[:n_train] // 10
y_val   = y[n_train:n_train+n_val] // 10
y_test  = y[n_train+n_val:] // 10

dist, idx = graph.distance_sklearn_metrics(X[0], k=3, metric='euclidean')
A = graph.adjacency(dist, idx).astype(np.float32)
graphs, perm = coarsening.coarsen(A, levels=3, self_connections=True)
X_train = coarsening.perm_data(X_train, perm)
X_test = coarsening.perm_data(X_test, perm)
L = [coarsening.laplacian(A, normalized=True) for A in X_train]

x_input = Input(shape=(X_train.shape[0],))
x = Reshape((X_train.shape[0],1))(x_input)
x = layers.GraphConvolution( 8, 2, 20, L[0])(x)
x = layers.GraphConvolution( 8, 4, 10, L[2])(x)
x = Flatten()(x)
x = Dense(66, activation='softmax')(x)

/usr/local/lib/python3.6/dist-packages/ChebyGCN/coarsening.py:198: RuntimeWarning: divide by zero encountered in true_divide
  tval = vv[rs+jj] * (1.0/weights[tid] + 1.0/weights[nid])
/usr/local/lib/python3.6/dist-packages/ChebyGCN/coarsening.py:198: RuntimeWarning: divide by zero encountered in true_divide
  tval = vv[rs+jj] * (1.0/weights[tid] + 1.0/weights[nid])
/usr/local/lib/python3.6/dist-packages/ChebyGCN/coarsening.py:198: RuntimeWarning: divide by zero encountered in true_divide
  tval = vv[rs+jj] * (1.0/weights[tid] + 1.0/weights[nid])


Layer 0: M_0 = |V| = 115384 nodes (100616 added),|E| = 43482 edges
Layer 1: M_1 = |V| = 57692 nodes (43026 added),|E| = 43465 edges
Layer 2: M_2 = |V| = 28846 nodes (14292 added),|E| = 43282 edges
Layer 3: M_3 = |V| = 14423 nodes (0 added),|E| = 43060 edges


ValueError: ignored